# **Operaciones de aprendizaje automatico (Grupo 10)**

**Equipo 19:**
  - Emmanuel Gallegos
  - José Esteban Jiménez
  - Carlos Ledezma
  - A01795195 | Jonathan Marmolejo
  - Salvador Mendoza
  - Emil Palomino

# **Declaración general de librerias**

In [41]:
import os
import numpy as np
import pandas as pd

# **1. Manipulación y preparación de datos**

**1a . Importar los conjuntos de datos con los que se trabajará.**

In [14]:
# Se importa la fuente de datos a trabajar.

from google.colab import drive
drive.mount('/content/drive')

import os
DIR = "/content/drive/My Drive/MLOps/"
os.chdir(DIR)

ObesityDataSet = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv",sep=',',header='infer')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Descripción del Dataset ObesityDataSet:**
Este conjunto de datos incluye información para la estimación de niveles de obesidad en individuos de los países de México, Perú y Colombia, en función de sus hábitos de alimentación y condición física.Los datos contienen **17 atributos y 2,111 registros.**
Los registros están etiquetados con la variable de **clase NObesity** (Nivel de Obesidad), que permite clasificar los datos utilizando los valores de *Peso Insuficiente, Peso Normal, Sobrepeso Nivel I, Sobrepeso Nivel II, Obesidad Tipo I. , Obesidad Tipo II y Obesidad Tipo III*. El 77% de los datos se generaron de forma sintética utilizando la herramienta Weka y el filtro SMOTE, el 23% de los datos se recogieron directamente de los usuarios a través de una plataforma web.

**1b. Realizar tareas de limpieza de datos, eliminando valores nulos o inconsistentes.**

In [28]:
# Identificamos las variables que componen el dataset y sus caracteristicas.
ObesityDataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [31]:
# Confirmamos que no tenga valores nulos.
ValNulos = ObesityDataSet.isnull().sum()
print(ValNulos)

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64


**NOTA:** Este dataset no contiene valores nulos.

**1c. Realizar transformaciones necesarias para preparar los datos para el análisis y modelado.**

In [49]:
# Clasificación de variables por tipo.

#numericas
numericas=ObesityDataSet.select_dtypes(include=['float64','int64'])
colnumericas=ObesityDataSet.select_dtypes(include=['float64','int64']).columns
#categoricas
categoricas=ObesityDataSet.select_dtypes(include=['object'])
colcategoricas=ObesityDataSet.select_dtypes(include=['object']).columns

categoricas.head().T
numericas.head().T

,0,1,2,3,4
Age,21.00,21.00,23.0,27.0,22.00
Height,1.62,1.52,1.8,1.8,1.78
Weight,64.00,56.00,77.0,87.0,89.80
FCVC,2.00,3.00,2.0,3.0,2.00
NCP,3.00,3.00,3.0,3.0,1.00
CH2O,2.00,3.00,2.0,2.0,2.00
FAF,0.00,3.00,2.0,2.0,0.00
TUE,1.00,0.00,1.0,0.0,0.00


In [46]:
# Identificación de valores en variables categoricas
for i in colcategoricas:
  print(i)
  print(ObesityDataSet[i].unique())

# Gender:                            Genero                                               | categorico.
# family_history_with_overweight:    Historial de familiares con sobrepeso                | Binario.
# FAVC:		                           ¿Consume alimentos ricos en calorías con frecuencia? |	Binario.
# CAEC:                              ¿Comes algún alimento entre comidas?		              |	Binario.
# SMOKE:	                           ¿Fumas?                                              |	Binario.
# SCC:	                             ¿Controlas las calorías que comes diariamente?       |	Binario.
# CALC:	                             ¿Cada cuando consumes alohol?                        | categorico.
# MTRANS:	                           ¿Qué trasporte utilizas habitualmente?		            | categorico.
# NObeyesdad:                        Nivel de obecidad                                    | categorico.

Gender
['Female' 'Male']
family_history_with_overweight
['yes' 'no']
FAVC
['no' 'yes']
CAEC
['Sometimes' 'Frequently' 'Always' 'no']
SMOKE
['no' 'yes']
SCC
['no' 'yes']
CALC
['no' 'Sometimes' 'Frequently' 'Always']
MTRANS
['Public_Transportation' 'Walking' 'Automobile' 'Motorbike' 'Bike']
NObeyesdad
['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


In [48]:
print(colnumericas)

#Para las variables numericas se estandarizará el valor según su definición

# Age:      Edad                                                       | continuo
# Height:   Estatura                                                   | continuo
# Weight:   Peso                                                       | continuo
# FCVC:     ¿Sueles comer vegetales en tus alimentos?	                 | entero
# NCP:      ¿Cuántas comidas principales haces al día?                 | entero
# CH2O:     ¿Cuanta agua tomas al día?                                 | entero
# FAF:      ¿Qué tan frecuente haces actividad física?                 | entero
# TUE:      ¿Que tan frecuente utilizas dispositivos electronicos?     | entero

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE'], dtype='object')


In [47]:
# Identificación de valores en variables numericas
for i in colnumericas:
  print(i)
  print(ObesityDataSet[i].unique())

Age
[21.       23.       27.       ... 22.524036 24.361936 23.664709]
Height
[1.62     1.52     1.8      ... 1.752206 1.73945  1.738836]
Weight
[ 64.        56.        77.       ... 133.689352 133.346641 133.472641]
FCVC
[2.       3.       1.       2.450218 2.880161 2.00876  2.596579 2.591439
 2.392665 1.123939 2.027574 2.658112 2.88626  2.714447 2.750715 1.4925
 2.205439 2.059138 2.310423 2.823179 2.052932 2.596364 2.767731 2.815157
 2.737762 2.568063 2.524428 2.971574 1.0816   1.270448 1.344854 2.959658
 2.725282 2.844607 2.44004  2.432302 2.592247 2.449267 2.929889 2.015258
 1.031149 1.592183 1.21498  1.522001 2.703436 2.362918 2.14084  2.5596
 2.336044 1.813234 2.724285 2.71897  1.133844 1.757466 2.979383 2.204914
 2.927218 2.88853  2.890535 2.530066 2.241606 1.003566 2.652779 2.897899
 2.483979 2.945967 2.478891 2.784464 1.005578 2.938031 2.842102 1.889199
 2.943749 2.33998  1.950742 2.277436 2.371338 2.984425 2.977018 2.663421
 2.753752 2.318355 2.594653 2.886157 2.967853 2.61983

In [70]:
# Se harán algunos ajustes en los valores para su mejor manejo

# Age:      Edad                                                       | continuo  | entero
# Height:   Estatura                                                   | continuo  | entero y dos digitos decimales
# Weight:   Peso                                                       | continuo  | entero y dos digitos decimales
# FCVC:     ¿Sueles comer vegetales en tus alimentos?	                 | entero    | entero                              | veces al día
# NCP:      ¿Cuántas comidas principales haces al día?                 | entero    | entero                              | veces al día
# CH2O:     ¿Cuanta agua tomas al día?                                 | entero    | entero y un digito decimal
# FAF:      ¿Qué tan frecuente haces actividad física?                 | entero    | entero                              | veces por semana
# TUE:      ¿Que tan frecuente utilizas dispositivos electronicos?     | entero    | entero                              | veces al día

# Se redondea a un entero y dos digitos
ObesityDataSet['Height']=ObesityDataSet['Height'].round(2)

# Se redondea el valor para posteriormente hacerlo entero
ObesityDataSet['Weight']=ObesityDataSet['Weight'].round()
ObesityDataSet['CH2O']=ObesityDataSet['CH2O'].round()
ObesityDataSet['Weight']=ObesityDataSet['Weight'].astype(int)
ObesityDataSet['CH2O']=ObesityDataSet['CH2O'].astype(int)

# Se convierte a valor entero
ObesityDataSet['Age']=ObesityDataSet['Age'].astype(int)
ObesityDataSet['FCVC']=ObesityDataSet['FCVC'].astype(int)
ObesityDataSet['NCP']=ObesityDataSet['NCP'].astype(int)
ObesityDataSet['FAF']=ObesityDataSet['FAF'].astype(int)
ObesityDataSet['TUE']=ObesityDataSet['TUE'].astype(int)

In [71]:
# Se comprueban resultados
print(ObesityDataSet['Height'].unique())

[1.62 1.52 1.8  1.78 1.5  1.64 1.72 1.85 1.65 1.77 1.7  1.93 1.53 1.71
 1.69 1.6  1.75 1.68 1.58 1.79 1.56 1.67 1.66 1.81 1.82 1.55 1.61 1.63
 1.76 1.57 1.88 1.87 1.89 1.74 1.83 1.92 1.73 1.84 1.91 1.48 1.59 1.54
 1.9  1.86 1.45 1.98 1.51 1.46 1.49 1.95 1.94]


In [72]:
# Identificación de valores en variables numericas
for i in colnumericas:
  print(i)
  print(ObesityDataSet[i].unique())

Age
[21 23 27 22 29 24 26 41 30 52 20 19 31 39 17 25 55 38 18 15 61 44 34 36
 32 35 40 45 33 51 56 28 16 37 14 42 50 43 46 47]
Height
[1.62 1.52 1.8  1.78 1.5  1.64 1.72 1.85 1.65 1.77 1.7  1.93 1.53 1.71
 1.69 1.6  1.75 1.68 1.58 1.79 1.56 1.67 1.66 1.81 1.82 1.55 1.61 1.63
 1.76 1.57 1.88 1.87 1.89 1.74 1.83 1.92 1.73 1.84 1.91 1.48 1.59 1.54
 1.9  1.86 1.45 1.98 1.51 1.46 1.49 1.95 1.94]
Weight
[ 64  56  77  87  90  53  55  68 105  80  99  60  66 102  78  82  70  50
  65  52  76  83  62  49  48  67  88  75  72  54  44  58  57  69  95 112
  85  45  91  86  93  84  42  63  51  61 109  94  59 130  79  96  98  74
 118  40  71 100  73 114 121 120  47  81 115 107 110 173  97 125  92 108
  39 113  89 104 127 111 138 165 133 134  43  41  46 101 106 103 119 117
 122 116 129 124 126 123 128 150 132 152 131 142 153 136 139 156 137 141
 144 149 155 161 151 140 135 154 143 147]
FCVC
[2 3 1]
NCP
[3 1 4 2]
CH2O
[2 3 1]
FAF
[0 3 2 1]
TUE
[1 0 2]


In [74]:
# Validación de dataset a trabajar:
ObesityDataSet.head(10)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21,1.62,64,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Normal_Weight
1,Female,21,1.52,56,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23,1.80,77,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Normal_Weight
3,Male,27,1.80,87,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Overweight_Level_I
4,Male,22,1.78,90,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29,1.62,53,no,yes,2,3,Sometimes,no,2,no,0,0,Sometimes,Automobile,Normal_Weight
6,Female,23,1.50,55,yes,yes,3,3,Sometimes,no,2,no,1,0,Sometimes,Motorbike,Normal_Weight
7,Male,22,1.64,53,no,no,2,3,Sometimes,no,2,no,3,0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24,1.78,64,yes,yes,3,3,Sometimes,no,2,no,1,1,Frequently,Public_Transportation,Normal_Weight
9,Male,22,1.72,68,yes,yes,2,3,Sometimes,no,2,no,1,1,no,Public_Transportation,Normal_Weight
